# Transfer Learning Guide: Fine-tune Pretrained Quantum Models

Save training time by starting from pretrained models.

**Time:** 15-20 minutes  
**Level:** Intermediate

## What is Transfer Learning?

Transfer learning allows you to:
1. Load a pretrained quantum model
2. Fine-tune it on your specific data
3. Save significant training time
4. Often achieve better performance

Think of it like starting with an experienced model rather than training from scratch.

In [ ]:
# Imports
from quantum_debugger.qml.transfer import PretrainedQNN
from quantum_debugger.qml.qnn import QuantumNeuralNetwork
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

## Method 1: Load from Model Zoo

The easiest way - use our pretrained models.

In [ ]:
# Load pretrained model (if available in model zoo)
# For demonstration, we'll train a base model first

# Create base model
print("Training base model...")
base_qnn = QuantumNeuralNetwork(n_qubits=4)
base_qnn.compile(optimizer='adam', loss='mse')

# Generate base training data
X_base, y_base = make_classification(
    n_samples=100, n_features=4, n_informative=4,
    n_redundant=0, n_classes=2, random_state=42
)

# Train base model
history = base_qnn.fit(X_base, y_base, epochs=20, verbose=0)
print(f"Base model trained. Final loss: {history['loss'][-1]:.4f}")

## Method 2: Create Pretrained Model from Existing QNN

In [ ]:
# Convert to PretrainedQNN
pretrained = PretrainedQNN.from_qnn(
    base_qnn,
    model_name='binary_classifier',
    dataset='synthetic',
    metadata={'n_qubits': 4, 'trained_epochs': 20}
)

print("Pretrained model created")
print(f"Model info: {pretrained.get_model_info()}")

## Fine-tuning: Standard Approach

Fine-tune all layers on new data.

In [ ]:
# Generate new task data (related but different)
X_new, y_new = make_classification(
    n_samples=80, n_features=4, n_informative=4,
    n_redundant=0, n_classes=2, random_state=123  # Different seed
)

X_train, X_test, y_train, y_test = train_test_split(
    X_new, y_new, test_size=0.25, random_state=42
)

print(f"New task - Train: {len(X_train)}, Test: {len(X_test)}")

In [ ]:
# Fine-tune (all layers)
print("Fine-tuning all layers...")
ft_history = pretrained.fine_tune(
    X_train, y_train,
    epochs=10,
    freeze_layers=0,  # 0 = no freezing
    verbose=0
)

print(f"Fine-tuned. Final loss: {ft_history['loss'][-1]:.4f}")

## Fine-tuning: Layer Freezing

Freeze early layers, only train final layers.

In [ ]:
# Create another pretrained model for comparison
pretrained_frozen = PretrainedQNN.from_qnn(
    base_qnn,
    model_name='binary_classifier_frozen',
    dataset='synthetic'
)

# Fine-tune with frozen layers
print("Fine-tuning with 2 layers frozen...")
ft_frozen_history = pretrained_frozen.fine_tune(
    X_train, y_train,
    epochs=10,
    freeze_layers=2,  # Freeze first 2 layers
    verbose=0
)

print(f"Fine-tuned (frozen). Final loss: {ft_frozen_history['loss'][-1]:.4f}")

## Compare: From Scratch vs Transfer Learning

In [ ]:
# Train from scratch for comparison
print("Training from scratch...")
scratch_qnn = QuantumNeuralNetwork(n_qubits=4)
scratch_qnn.compile(optimizer='adam', loss='mse')
scratch_history = scratch_qnn.fit(X_train, y_train, epochs=10, verbose=0)

print(f"From scratch. Final loss: {scratch_history['loss'][-1]:.4f}")

In [ ]:
# Evaluate all models
models = {
    'Transfer (All)': pretrained,
    'Transfer (Frozen)': pretrained_frozen,
    'From Scratch': scratch_qnn
}

results = {}
for name, model in models.items():
    if isinstance(model, PretrainedQNN):
        preds = model.predict(X_test)
    else:
        preds = model.predict(X_test)
    
    pred_labels = (preds > 0.5).astype(int).flatten()
    accuracy = np.mean(pred_labels == y_test)
    results[name] = accuracy
    print(f"{name}: {accuracy:.3f}")

## Visualize Training Progress

In [ ]:
plt.figure(figsize=(12, 5))

# Loss comparison
plt.subplot(1, 2, 1)
plt.plot(ft_history['loss'], label='Transfer (All)', linewidth=2)
plt.plot(ft_frozen_history['loss'], label='Transfer (Frozen)', linewidth=2)
plt.plot(scratch_history['loss'], label='From Scratch', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# Accuracy comparison
plt.subplot(1, 2, 2)
plt.bar(results.keys(), results.values())
plt.ylabel('Accuracy')
plt.title('Final Test Accuracy')
plt.ylim([0, 1])
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Save and Load Models

In [ ]:
# Save pretrained model
pretrained.save('my_quantum_model.pkl')
print("Model saved to my_quantum_model.pkl")

# Load later
loaded_model = PretrainedQNN.load('my_quantum_model.pkl')
print("Model loaded successfully")

# Verify it works
test_pred = loaded_model.predict(X_test[:5])
print(f"\nTest predictions: {test_pred.flatten()}")

## Best Practices

### When to use Transfer Learning:
1. **Limited data** - You have small training set
2. **Similar tasks** - New task related to pretrained model's task
3. **Fast prototyping** - Need quick results
4. **Resource constrained** - Limited training time/compute

### Layer Freezing Strategy:
- **Freeze more layers** when new data is very similar to original
- **Freeze fewer layers** when new task is quite different
- **No freezing** when you have lots of new data

### Tips:
1. Always compare with training from scratch
2. Experiment with different freeze levels
3. Use lower learning rate for fine-tuning
4. Monitor validation performance

## Next Steps

Try:
1. Use your own dataset
2. Experiment with freeze_layers parameter
3. Compare training times
4. Build your own model zoo

Other notebooks:
- `01_quickstart_automl.ipynb` - AutoML basics
- `03_hardware_deployment.ipynb` - Real quantum hardware
- `04_advanced_optimization.ipynb` - Circuit optimization
- `05_benchmarking_qml_vs_classical.ipynb` - Performance comparison